In [1]:
using Flux
using Symbolics
using SphericalHarmonics
using CUDA
using NNlib

using TensorCast

# Disabling slow GPU indexing
CUDA.allowscalar(false)

using BenchmarkTools
using ProgressMeter

include("utils.jl")
include("Spherical.jl");

### Defining Data

In [4]:
# Define Tetris Shapes
tetris = [[0 0 0; 0 0 1; 1 0 0; 1 1 0],  # chiral_shape_1
          [0 0 0; 0 0 1; 1 0 0; 1 -1 0], # chiral_shape_2
          [0 0 0; 1 0 0; 0 1 0; 1 1 0],  # square
          [0 0 0; 0 0 1; 0 0 2; 0 0 3],  # line
          [0 0 0; 0 0 1; 0 1 0; 1 0 0],  # corner
          [0 0 0; 0 0 1; 0 0 2; 0 1 0],  # T
          [0 0 0; 0 0 1; 0 0 2; 0 1 1],  # zigzag
          [0 0 0; 1 0 0; 1 1 0; 2 1 0]]  # L

4×3 Matrix{Int64}:
 0  0  0
 0  0  1
 1  0  0
 1  1  0

### Testing Network Definition

In [5]:
include("TFNLayers.jl")

In [5]:
# testing R
centers = range(0f0, 3.5f0; length=4) |> collect
r_test = RLayer(centers) |> gpu

RLayer{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}, Vector{Float32}, Float32}(Float32[-0.48342064, -0.5924541, -0.46601388, -0.4184361], Float32[0.0, 1.1666666, 2.3333333, 3.5], 0.875f0)

In [24]:
n_samples = 100
n_points = 4
ℓi, ℓf, ℓos = 2, 1, [2]
total_out = sum(2ℓo + 1 for ℓo in ℓos)

centers = range(0f0, 3.5f0; length=4) |> collect
c_test = CLayer(ℓi, ℓf, ℓos, centers) |> gpu

xs = rand(Float32, (n_points, 3, n_samples))
xss = pairwise_rs(xs)
rss = cart_to_sph(xss) |> gpu

V = ones(Float32, (2ℓi+1, n_points, n_samples)) |> gpu
#V = ones(Float32, (1, 4))
outs_fake = rand(Float32, (n_points, n_samples, total_out)) |> gpu

C_out = c_test(rss, V)

4×100×5 CuArray{Float32, 3, CUDA.Mem.DeviceBuffer}:
[:, :, 1] =
 -0.0203455  -0.0605927   0.0964717  …   0.0292929  -0.679569   -0.416552
 -0.134659    0.214497    0.0648599     -0.0244393   0.0165871  -0.359965
 -0.563291   -0.381318   -0.369426      -0.249671   -0.2314      0.22246
 -0.105303   -0.596185   -0.615504      -0.578781    0.0707832  -0.269542

[:, :, 2] =
 -0.170749    0.027444   0.29558    …   0.319952  -0.673482  -0.32275
  0.123802    0.100373  -0.0630427     -0.292839   0.174428   0.0463389
 -0.425657   -0.199426  -0.0284927     -0.256734   0.266216   0.0555241
  0.0608048  -0.34019   -0.615844      -0.182178  -0.178961  -0.190913

[:, :, 3] =
 -0.217043    0.0893105   0.314773  …   0.373602  -0.506208  -0.172507
  0.32785     0.0617457  -0.107808     -0.337114   0.239798   0.336949
 -0.198522   -0.10526     0.236678     -0.156087   0.540715  -0.0072757
  0.0877144  -0.0457968  -0.443643      0.119599  -0.274306  -0.157166

[:, :, 4] =
 -0.223508   0.134788    0.27620

In [9]:
optim = Flux.setup(Flux.Adam(0.1), c_test)

losses = []
@showprogress for epoch in 1:400
    loss, grads = Flux.withgradient(c_test) do f
        # Evaluate model and loss inside gradient context:
        y_hat = f(rss, V)
        Flux.mse(y_hat, outs_fake)
    end
    Flux.update!(optim, c_test, grads[1])
    push!(losses, loss)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:09


### Testing Increasing Dimensionality

In [3]:
n_samples = 6
n_points = 4

xs = rand(Float32, (n_points, 3, n_samples))
xss = pairwise_rs(xs)
rss = cart_to_sph(xss)

ℓ = 2
Ys = generate_Yℓms(ℓ)
yss = rand(Float32, (n_points, n_points, n_samples, 2ℓ+1))

rss_gpu = rss |> gpu
yss_gpu = yss |> gpu

centers = range(0f0, 3.5f0; length=4) |> collect
f_test = FLayer(Ys, centers) |> gpu

f_test(rss_gpu)

4×4×6×5 CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}:
[:, :, 1, 1] =
 -0.0          0.00754981  -0.0499345  -0.116065
  0.00754981  -0.0         -0.213974   -0.329704
 -0.0499345   -0.213974    -0.0         0.545569
 -0.116065    -0.329704     0.545569   -0.0

[:, :, 2, 1] =
 -0.0        0.232802   0.160161    0.262934
  0.232802  -0.0       -0.229806    0.456423
  0.160161  -0.229806  -0.0         0.00174865
  0.262934   0.456423   0.0017487  -0.0

[:, :, 3, 1] =
 -0.0        -0.370857   0.226216    -0.0809055
 -0.370857   -0.0       -0.503037    -0.22966
  0.226216   -0.503037  -0.0          0.00735487
 -0.0809054  -0.22966    0.00735485  -0.0

[:, :, 4, 1] =
 -0.0       -0.11373     0.145963   0.258541
 -0.11373   -0.0         0.113866  -0.0867647
  0.145963   0.113866   -0.0        0.141403
  0.258541  -0.0867647   0.141403  -0.0

[:, :, 5, 1] =
 -0.0        0.166011  -0.113239   0.15408
  0.166011  -0.0        0.425438  -0.265191
 -0.113239   0.425438  -0.0        0.450372
  0.15408

In [4]:
optim = Flux.setup(Flux.Adam(0.01), f_test)

# Testing gradient
losses = []
@showprogress for epoch in 1:400
    loss, grads = Flux.withgradient(f_test) do f
        # Evaluate model and loss inside gradient context:
        y_hat = f(rss_gpu)
        Flux.mse(y_hat, yss_gpu)
    end
    Flux.update!(optim, f_test, grads[1])
    push!(losses, loss)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:36
